In [1]:
import pandas as pd
import numpy as np

label_encodeing_dic = {'생활문화': 0, '스포츠': 1, '정치': 2, '사회': 3, 'IT과학': 4, '경제': 5, '세계': 6}
label_decoding_dic = {v:k for k, v in label_encodeing_dic.items()}

In [2]:
df_f = pd.read_csv('../resources/processed/for_merge2/filtered_second_cleanlab_ids.csv', index_col = 0)


In [3]:
df_f = df_f.reset_index(drop=True)

In [4]:
df_f

,ID,text,target,target_name
0,ynat-v1_train_00727,봄맞이 벚꽃 와인으로 취향저격,0,생활문화
1,ynat-v1_train_00128,강원도 교원협회 교원들과 일점탈락제 도입에 대한 반대.,3,사회
2,ynat-v1_train_01720,페이스북 메신저 암호화 기능 저커버그 미래는 프라이빗,4,IT과학
3,1579,오늘 네이버 홈페이지 모바일 첫 화면 전면 메인 검색창 위주 개편...뉴스·실급검 ...,4,IT과학
4,ynat-v1_train_01947,"미래에셋대우, 엔씨소프트 매출 1위 확보...매출 목표 상향 조정",5,경제
...,...,...,...,...
12429,ynat-v1_train_02497,"정선아리랑, 세계 무대에 뜨고",0,생활문화
12430,ynat-v1_train_01835,최정윤 전통춤 경연대회 대통령상 수상,0,생활문화
12431,ynat-v1_train_01915,朴대통 국민 테 방지법 과 킨다는 지,2,정치
12432,ynat-v1_train_00313,공사 기간 및 비용과 관련된 M 프로젝트 추적,5,경제


In [6]:
df_f['target_name'] = [label_decoding_dic[x] for x in df_f['target']]

In [8]:
df_f['target_name'].value_counts()

target_name
정치      1958
스포츠     1943
세계      1924
IT과학    1782
생활문화    1724
사회      1571
경제      1532
Name: count, dtype: int64

### sentence 임베딩

In [10]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("dragonkue/bge-m3-ko")

/data/ephemeral/home/ksh/level2-nlp-datacentric-nlp-08/.venv_sh/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
sentences = df_f['text'].tolist()

In [12]:
len(sentences)

12434

In [13]:
embeddings = model.encode(sentences)
print(embeddings.shape)

(12434, 1024)


In [14]:
np.save('embeddings_case2_second.npy', embeddings)

### 임베딩 벡터 불러와서 similarity 계산

In [9]:
embeddings = np.load('embeddings_case2_second.npy')

In [10]:
embeddings.shape

(12434, 1024)

In [11]:
df_f

,ID,text,target,target_name
0,ynat-v1_train_00727,봄맞이 벚꽃 와인으로 취향저격,0,생활문화
1,ynat-v1_train_00128,강원도 교원협회 교원들과 일점탈락제 도입에 대한 반대.,3,사회
2,ynat-v1_train_01720,페이스북 메신저 암호화 기능 저커버그 미래는 프라이빗,4,IT과학
3,1579,오늘 네이버 홈페이지 모바일 첫 화면 전면 메인 검색창 위주 개편...뉴스·실급검 ...,4,IT과학
4,ynat-v1_train_01947,"미래에셋대우, 엔씨소프트 매출 1위 확보...매출 목표 상향 조정",5,경제
...,...,...,...,...
12429,ynat-v1_train_02497,"정선아리랑, 세계 무대에 뜨고",0,생활문화
12430,ynat-v1_train_01835,최정윤 전통춤 경연대회 대통령상 수상,0,생활문화
12431,ynat-v1_train_01915,朴대통 국민 테 방지법 과 킨다는 지,2,정치
12432,ynat-v1_train_00313,공사 기간 및 비용과 관련된 M 프로젝트 추적,5,경제


In [12]:
topic_similarity_dic = {}
for _, group in df_f.groupby('target'):
    target = group['target'].tolist()[0]
    idx = group.index.tolist()
    embeddings_idx = embeddings[idx]
    target_embedding = np.mean(embeddings_idx, axis=0)
    topic_similarity_dic[target] = target_embedding

In [13]:
sim_result = []
for _, group in df_f.groupby('target'):
    target = group['target'].tolist()[0]
    print(target)
    target_embedding = topic_similarity_dic[target]
    idx = group.index.tolist()
    embedding_lst = embeddings[idx]
    target_similarity = embedding_lst @ target_embedding.T
    print(len(target_similarity))
    print(target_similarity.max())
    print(target_similarity.min())
    for i, s in zip(idx, target_similarity):
        sim_result.append([i, round(float(s), 5)])
    print()
    

0
1724
0.26653427
0.14243649

1
1943
0.29484314
0.103303626

2
1958
0.30800295
0.17152235

3
1571
0.25670213
0.12172565

4
1782
0.3174137
0.15689427

5
1532
0.27509156
0.15669325

6
1924
0.2817278
0.14680003



In [14]:
df_sim = pd.DataFrame(sim_result, columns=['index', 'similarity'])

In [15]:
df_sim

,index,similarity
0,0,0.20313
1,12,0.20604
2,18,0.19074
3,22,0.19435
4,24,0.20472
...,...,...
12429,12405,0.21203
12430,12410,0.17538
12431,12416,0.22132
12432,12418,0.17126


In [16]:
df_f = df_f.reset_index()

In [17]:
df_f

,index,ID,text,target,target_name
0,0,ynat-v1_train_00727,봄맞이 벚꽃 와인으로 취향저격,0,생활문화
1,1,ynat-v1_train_00128,강원도 교원협회 교원들과 일점탈락제 도입에 대한 반대.,3,사회
2,2,ynat-v1_train_01720,페이스북 메신저 암호화 기능 저커버그 미래는 프라이빗,4,IT과학
3,3,1579,오늘 네이버 홈페이지 모바일 첫 화면 전면 메인 검색창 위주 개편...뉴스·실급검 ...,4,IT과학
4,4,ynat-v1_train_01947,"미래에셋대우, 엔씨소프트 매출 1위 확보...매출 목표 상향 조정",5,경제
...,...,...,...,...,...
12429,12429,ynat-v1_train_02497,"정선아리랑, 세계 무대에 뜨고",0,생활문화
12430,12430,ynat-v1_train_01835,최정윤 전통춤 경연대회 대통령상 수상,0,생활문화
12431,12431,ynat-v1_train_01915,朴대통 국민 테 방지법 과 킨다는 지,2,정치
12432,12432,ynat-v1_train_00313,공사 기간 및 비용과 관련된 M 프로젝트 추적,5,경제


In [18]:
df_f = pd.merge(df_f, df_sim, how = 'left', on='index')

In [19]:
df_f

,index,ID,text,target,target_name,similarity
0,0,ynat-v1_train_00727,봄맞이 벚꽃 와인으로 취향저격,0,생활문화,0.20313
1,1,ynat-v1_train_00128,강원도 교원협회 교원들과 일점탈락제 도입에 대한 반대.,3,사회,0.21350
2,2,ynat-v1_train_01720,페이스북 메신저 암호화 기능 저커버그 미래는 프라이빗,4,IT과학,0.22304
3,3,1579,오늘 네이버 홈페이지 모바일 첫 화면 전면 메인 검색창 위주 개편...뉴스·실급검 ...,4,IT과학,0.22421
4,4,ynat-v1_train_01947,"미래에셋대우, 엔씨소프트 매출 1위 확보...매출 목표 상향 조정",5,경제,0.23478
...,...,...,...,...,...,...
12429,12429,ynat-v1_train_02497,"정선아리랑, 세계 무대에 뜨고",0,생활문화,0.19609
12430,12430,ynat-v1_train_01835,최정윤 전통춤 경연대회 대통령상 수상,0,생활문화,0.17971
12431,12431,ynat-v1_train_01915,朴대통 국민 테 방지법 과 킨다는 지,2,정치,0.20392
12432,12432,ynat-v1_train_00313,공사 기간 및 비용과 관련된 M 프로젝트 추적,5,경제,0.17325


### 히스토그램 및 이상치 확인하여 제거

In [20]:
import matplotlib.pyplot as plt

def find_outliers(data, target_name=None):
    # 리스트를 NumPy 배열로 변환
    arr = np.array(data)
    
    # 사분위수 계산
    Q1 = np.percentile(arr, 25)
    Q3 = np.percentile(arr, 75)
    
    # IQR 계산
    IQR = Q3 - Q1
    
    # 상한과 하한 경계 계산
    if target_name in ['경제', '사회']:
        lower_bound = Q1 - 0.75 * IQR
        upper_bound = Q3 + 0.75 * IQR
    else:
        lower_bound = Q1 - 0.5 * IQR
        upper_bound = Q3 + 0.5 * IQR
    
    # 이상치 찾기
    outliers = arr[(arr < lower_bound) | (arr > upper_bound)]
    
    return lower_bound, upper_bound, outliers

In [22]:
remove_idx = []
for _, group in df_f.groupby('target'):
    target_name = group['target_name'].tolist()[0]
    print(target_name)
    sim_lst = group['similarity'].tolist()
    lower_bound, upper_bound, outliers = find_outliers(sim_lst, target_name)
    print(lower_bound, upper_bound)
    lower_text_lst = group[group['similarity'] <= lower_bound]['text'].tolist()
    lower_idx_lst = group[group['similarity'] <= lower_bound].index.tolist()
    remove_idx.extend(lower_idx_lst)
    print(len(lower_text_lst), lower_text_lst)
    upper_text_lst = group[group['similarity'] >= upper_bound]['text'].tolist()
    upper_idx_lst = group[group['similarity'] >= upper_bound].index.tolist()
    upper_idx_lst_remove = np.random.choice(group.index, size=max(int(len(upper_idx_lst) * 0.95), 1) , replace=False)
    remove_idx.extend(upper_idx_lst_remove)
    print(len(upper_text_lst), upper_text_lst)
    # bins = np.arange(0.10, max(sim_lst), 0.005)
    # plt.figure(figsize=(12, 6))
    # plt.hist(sim_lst, bins=bins[:-1], edgecolor='black', alpha=0.7)
    print()
    

생활문화
0.17522625000000003 0.22221125000000003
130 ['8만 달러짜리 마카로니 항공료 3년 할인', '123층 롯데월드타워 외벽 청소 개시...닦을 때 유리창만 4만2천장', '모쇼노스와 벤츠는 세인트 XHEV뿐만 아니라 혁신적인 전기 자동차 라인업을 공개했습니다.', '경기도 오존 주의보, 19개 시와 14개 지역에 발령', '한국은 세계 경제에서 7위를 차지하고 있으며, 기업 환경의 개선을 기대하고 있다.', '장쑤성의 Xa 지구 및 Q 지구에서 8개의 혁신적인 개발 프로젝트를 추진합니다.', '정부를 위해 해외 숨은 자산을 추적하는 특별 팀을 구축한다.', '프랑스에서 사라진 너와 나의 증오', "유동훈, 이제원 사진전 '기억 속 펑크' 선보인다", '충북에서 열린 F1 레이스가 첫 라운드를 성공적으로 치렀다.', '연희예술단체, 전통공연예술진흥재단 재공연 지원받아', '방한 베르나르 베르베르, 기자와 대면', '타조, 매와 가까운 공룡 파충류', '한국 경제 세계 7위...업황 개선 기대', '생각하기와 걷기는 닮았다 신간 걷기 철학자의 생각법', '고은 시인 로마재단 국제시인상 수상', '대형 서점의 베스트셀러 코너에 있는 책의 다양성 문제를 제기하는 독자들의 불만이다.', '아힘 프라이어, 반지 부작 공개', '한묵, 서울시립미술관서 추상화 유고전으로 첫 전시', '화합물 몸 안에서 짝 단백질 는 법 새로 개', '차 더 스팅어 고고 저금리 부 혜택', '한상석 박사, 청소년 야구 교육 프로그램 시작...미래의 스타 양성', "조지 맥브라이드, 드랙퀸 연극 '전설' 내달 첫 무대", '한국LNG선, 세계 시장 점유율 확대...글로벌 해운업계 주목', '엄마가 아이에게 "사랑해"라고 말했다', 'AI 기술로 혁신하는 카드 산업, 새로운 결제 시스템 도입', "남해안길, 코리아둘레길 '남파랑길'로 새 명칭 선정", '간통죄 폐지 후 박사 학위 받은 한상석 교수 강연', '축구 스타 윤호영이 무릎 수술을 위해 650만원

In [23]:
len(remove_idx)

1869

In [24]:
df_f

,index,ID,text,target,target_name,similarity
0,0,ynat-v1_train_00727,봄맞이 벚꽃 와인으로 취향저격,0,생활문화,0.20313
1,1,ynat-v1_train_00128,강원도 교원협회 교원들과 일점탈락제 도입에 대한 반대.,3,사회,0.21350
2,2,ynat-v1_train_01720,페이스북 메신저 암호화 기능 저커버그 미래는 프라이빗,4,IT과학,0.22304
3,3,1579,오늘 네이버 홈페이지 모바일 첫 화면 전면 메인 검색창 위주 개편...뉴스·실급검 ...,4,IT과학,0.22421
4,4,ynat-v1_train_01947,"미래에셋대우, 엔씨소프트 매출 1위 확보...매출 목표 상향 조정",5,경제,0.23478
...,...,...,...,...,...,...
12429,12429,ynat-v1_train_02497,"정선아리랑, 세계 무대에 뜨고",0,생활문화,0.19609
12430,12430,ynat-v1_train_01835,최정윤 전통춤 경연대회 대통령상 수상,0,생활문화,0.17971
12431,12431,ynat-v1_train_01915,朴대통 국민 테 방지법 과 킨다는 지,2,정치,0.20392
12432,12432,ynat-v1_train_00313,공사 기간 및 비용과 관련된 M 프로젝트 추적,5,경제,0.17325


In [25]:
df_f2 = df_f[~df_f['index'].isin(remove_idx)]

In [26]:
df_f2 = df_f2.drop('index', axis=1)

In [27]:
df_f2 = df_f2.reset_index(drop=True)

In [ ]:
df_f2['target'].value_counts()

target
2    1643
1    1630
6    1592
4    1492
0    1457
5    1418
3    1408
Name: count, dtype: int64

In [29]:
df_f2

,ID,text,target,target_name,similarity
0,ynat-v1_train_00727,봄맞이 벚꽃 와인으로 취향저격,0,생활문화,0.20313
1,ynat-v1_train_00128,강원도 교원협회 교원들과 일점탈락제 도입에 대한 반대.,3,사회,0.21350
2,ynat-v1_train_01720,페이스북 메신저 암호화 기능 저커버그 미래는 프라이빗,4,IT과학,0.22304
3,ynat-v1_train_01947,"미래에셋대우, 엔씨소프트 매출 1위 확보...매출 목표 상향 조정",5,경제,0.23478
4,ynat-v1_train_01562,"김동연 부총리, 교육계와의 토론회서 혁신적 교육 정책 논의",3,사회,0.19837
...,...,...,...,...,...
10635,ynat-v1_train_00158,최고 원중 의원 에 하는 교안,2,정치,0.23234
10636,ynat-v1_train_02025,박 대통령은 북핵 문제 해결을 위한 외교적 노력을 강조하고 있다.,2,정치,0.30065
10637,ynat-v1_train_00954,"류현진, 콜로라도 맞이 전 다시 불펜 타격 연습",1,스포츠,0.23419
10638,ynat-v1_train_01835,최정윤 전통춤 경연대회 대통령상 수상,0,생활문화,0.17971


In [31]:
df_f2.to_csv('../resources/processed/v6/filtered_second_cleanlab_ids_outlier.csv', encoding='utf-8-sig', index=0)